# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
from openai import OpenAI
from IPython.display import display, Markdown, update_display

In [2]:
# constants
# MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

class LLM_MODEL:

    def ask_model(self, sys_prompt, usr_prompt):
        model_url =  'http://localhost:11434/v1/'
        client = OpenAI(base_url=model_url, api_key='ollama')
        msg = [{'role':'system', 'content':sys_prompt},{'role':'user', 'content':usr_prompt}]
        response = client.chat.completions.create(model=MODEL_LLAMA, messages=msg)
        return response.choices[0].message.content

    def ask_model_stream(self, sys_prompt, usr_prompt):
        model_url =  'http://localhost:11434/v1/'
        client = OpenAI(base_url=model_url, api_key='ollama')
        msg = [{'role':'system', 'content':sys_prompt},{'role':'user', 'content':usr_prompt}]
        stream = client.chat.completions.create(model=MODEL_LLAMA, messages=msg, stream=True)
        return stream

model = LLM_MODEL()

In [4]:
# Task 1: Tight Speed

sys_prompt = 'You are a helpful assistant who helps me understand technical questions.\n'
usr_prompt = 'It takes Alex 2 hours to travel a distance of 3 kms. What is the speed of Alex?'

resp = model.ask_model(sys_prompt, usr_prompt)
display(Markdown(resp))

To find the speed of Alex, we need to use the formula:

Speed = Distance / Time

We know the distance (3 kms) and the time it took for the journey (2 hours).

First, let's convert the distance from kilometers to meters: 1 km = 1000 meters, so:
Distance (in meters) = 3 km × 1000 m/km = 3000 meters

Now we can plug in the values:

Speed = Distance / Time
= 3000 meters / 2 hours
= 1500 meters-per-hour

To make it more readable, let's convert this to kilometers per hour (km/h):
1 meter = 0.001 km (to convert meters to kilometers), so:
= 1500 m ÷ 1000 = 1.5 km

Therefore, Alex's speed is 1.5 kilometers per hour.

In [5]:
# Task 2: Travel the world in X days?

sys_prompt = 'You are a helpful assistant who helps me understand technical questions.\n'
usr_prompt = 'There are many cities in our world. Can you tell me how to travel the whole world in least number of days ?'

resp = model.ask_model(sys_prompt, usr_prompt)
display(Markdown(resp))

Traveling around the world is an exciting adventure! To help you minimize your travel time, I'll provide a general outline of the most efficient way to cover all continents and major cities.

**The Most Efficient Route:**

1. Start from North America (USA or Canada) and head east:
	* Fly from Los Angeles to Dubai
	* From Dubai, take a Middle Eastern flight to Istanbul, Turkey
2. Next, enter Europe by flying back west from Istanbul:
	* Take trains and buses between major European cities like Berlin, Prague, Vienna, etc.
3. Head south into Asia:
	* From Eastern Europe, fly to Delhi or Mumbai in India
	* Then, take flights to Southeast Asian countries like Bangkok (Thailand), Jakarta (Indonesia), or Kuala Lumpur (Malaysia)
4. Cross into Africa and visit major cities:
	* Fly from Southeast Asia to Cairo, Egypt
	* Explore North African countries like Morocco, Tunisia, and Algeria
5. From Africa, head north into Europe again:
	* Fly back to Western European countries like London (UK), Paris (France), or Amsterdam (Netherlands)
6. Finally, enter South America from Europe:
	* Take flights from European cities to Buenos Aires (Argentina) or Rio de Janeiro (Brazil)

**Tips and Considerations:**

1. **Fly through major hubs:** Using airports like Dubai, Istanbul, Cairo, Bangkok, and Singapore will simplify your journey.
2. **Choose efficient airlines:** Look for ultra-low-cost carriers, budget airlines, or hybrid models that offer competitive prices.
3. **Plan smart connections:** Research flight schedules, layovers, and travel restrictions to minimize delays.
4. **Use visa-free policies:** Make the most of visa exemptions where possible, like e-Visas for India, Mexico, and some African countries.
5. **Health insurance:** Check if your travel insurance covers medical care abroad.

**Time Estimates:**

* Assuming a moderate pace (some planning, but no frills), you can cover around 10-15 major cities in 2-3 months with decent connections and layovers.
* However, this pace is dependent on your personal interests, budget, and flexibility. Be prepared to adjust based on changing circumstances or unexpected delays.

**Additional Tips:**

1. Consider the weather, peak tourist seasons, and holidays when planning your trip.
2. Bring essential documents like passports, visas (if required), travel insurance, and health certificates.
3. Research local regulations, COVID-19 guidelines, and vaccinations before traveling to specific countries.

Keep in mind that this outline is a general suggestion, and actual times will vary depending on your start date, flight options, visa processing, and additional activities (like snorkeling or hiking) you'd like to incorporate.

Is there anything else I can help with?

In [27]:
# Task 3: Generate Code for task 4 to scrap some webpages

sys_prompt = 'You are a coding expert who generates python code for given problem.\n'
usr_prompt = 'Given a website URL, I want to a python function to get the contents of the webpage, and another function to parse all links in the given webpage text.'

resp = model.ask_model(sys_prompt, usr_prompt)
print(resp)

Here's an example implementation using Python with the `requests` library to fetch the webpage content and `BeautifulSoup` for HTML parsing.

### Install Required Libraries
```bash
pip install requests beautifulsoup4
```

### Code Implementation

```python
import requests
from bs4 import BeautifulSoup

def get_webpage_content(url):
    """
    Fetches the contents of a website.
    
    Args:
        url (str): URL of the webpage.
    
    Returns:
        str: HTML content of the webpage.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching webpage: {e}")
        return None

def parse_links(html_content, base_url=""):
    """
    Parses links from a given HTML content.
    
    Args:
        html_content (str): HTML content of the webpage.
        base_url (str): Base URL to construct relative link

In [6]:
# Scrap some webpages

import requests
from bs4 import BeautifulSoup

def get_webpage_content(url):
    """
    Fetches the contents of a website.
    
    Args:
        url (str): URL of the webpage.
    
    Returns:
        str: HTML content of the webpage.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching webpage: {e}")
        return None

def parse_links(html_content, base_url=""):
    """
    Parses links from a given HTML content.
    
    Args:
        html_content (str): HTML content of the webpage.
        base_url (str): Base URL to construct relative link URLs. Defaults to "".
    
    Returns:
        list: List of extracted URLs.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    links = []

    for tag in soup.find_all('a'):
        href = tag.get('href')

        # Handle absolute and relative URLs
        if not href or href.startswith('/'):
            url = ""
        else:
            if 0 and base_url:
                url = f"{base_url}{href}"
            else:
                url = href

        if url.startswith('https:/'):
            links.append(url)

    return links


In [7]:
# Task 4: Make a brochure using the web-content

# Example usage
webname, url = 'Huggingface', "http://www.huggingface.co"

html_content = get_webpage_content(url)
links = parse_links(html_content, url)

print("Extracted Links:")
content = f'Link:{url} -> Content:{html_content}\n'
for link in links:
    print(link)
    html_content = get_webpage_content(url)
    content += f'Link:{link} -> Content:{html_content}\n'

sys_prompt = 'You are a helpful assistant who helps me create a brochure for a website.\n'
usr_prompt = f'You are given the contents for a few pages for the website of {webname} following next line.\n' + \
             content + \
             'Use this information to give the brochure for this company.\n'

stream = model.ask_model_stream(sys_prompt, usr_prompt)

response = ''
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)


Extracted Links:
https://endpoints.huggingface.co
https://apply.workable.com/huggingface/
https://discuss.huggingface.co
https://status.huggingface.co/
https://github.com/huggingface
https://twitter.com/huggingface
https://www.linkedin.com/company/huggingface/


Here's a possible brochure design and content based on the code snippet provided:

**[Cover Page]**

* Title: Hugging Face
* Tagline: Building sustainable AI models for everyone
* Background image: A gradient background with a collage of diverse images, likely representing people from different cultures and backgrounds working together.

**[Inside Pages]**

**[Page 1: About Us]**

* Headline: Discover the Power of AI Models on Hugging Face
* Text: Hugging Face is a leading open-source platform for natural language processing (NLP) models. Our mission is to empower researchers, developers, and businesses to build and use high-quality AI models that can be applied in various industries.
* Image: A group photo of the Hugging Face team

**[Page 2: Models]**

* Headline: Explore the Largest Collection of Pre-Trained NLP Models
* Text: Our model portal offers over 200 pre-trained models, covering a wide range of tasks such as sentiment analysis, entity recognition, and language translation.
* Features:
 + Model browsing by task or dataset
 + Filtering by accuracy, accuracy distribution, weights, and more
	+ Training from scratch options for advanced users
* Image: A screenshot of the model portal with a random selection of models

**[Page 3: Datasets]**

* Headline: Tap into a Universe of High-Quality Datasets for Model Training
* Text: Hugging Face's dataset repository includes over 1 million datasets, covering various domains such as text analysis, speech recognition, and sentiment analysis.
* Features:
 + Dataset browsing by domain or type
 + Filtering by size, download time, license, and more
	+ Data augmentation options
* Image: A screenshot of the dataset repository with a random selection of datasets

**[Page 4: Spaces]**

* Headline: Collaborate on Research Projects and Share Models
* Text: Our shared model hosting platform allows researchers to collaborate on open-source projects, share models, and receive feedback from community members.
* Features:
 + Project creation options for collaboration
	+ Model sharing and download
	+ Discussion forums for feedback and support
* Image: A screenshot of the spaces dashboard with a selected project

**[Page 5: Changelog]**

* Headline: Stay Up-to-Date on the Latest Hugging Face Features
* Text: Get notified about new model releases, dataset updates, and feature enhancements through our changelog.
* Format:
	+ List of recent features and bug fixes with brief descriptions
	+ Links to documentation or demo models for some features
	+ Option to subscribe to notifications via email
* Image: A screenshot of the changelog as it appears on a mobile device

**[Back Cover]**

* Call-to-Action (CTA): Sign up for our newsletter and get started with Hugging Face today!
* Text: "Unlock the power of AI models for everyone. Subscribe to our newsletter for news, tutorials, and special offers."
* Background image: The same collage as the cover page.

**Additional Materials**

* Business card template with contact information
* Letterhead with the company's logo
* One-page brochure for each specific product or feature (e.g., Model Card, Dataset Card)

Note that this is just a rough outline and can be customized to fit your specific needs. The image and design elements used should be consistent throughout the brochure and online presence.